In [21]:
import pandas as pd
import requests
import time
import re
import unicodedata
import json
from datetime import datetime

In [2]:
stations_df = pd.read_csv('stations_info.csv')

In [3]:
stations_df.head()

,city,station_name,station_id,station_lat,station_lon,station_url
0,Jelenia Góra,"Jelenia Góra - Ogińskiego, Poland",8135,50.913470,15.765659,poland/dolnoslaskie/jelenia-gora-oginskiego
1,Biała Podlaska,"Biała Podlaska, Poland",6479,52.038713,23.144503,poland/lublin/biala-podlaska
2,Białystok,"Białystok-Miejska, Poland",6520,53.126815,23.156370,poland/bialystok/bialystok-miejska
3,Bielsko-Biała,"Bielsko-Biała, ul. Partyzantów, Poland",10544,49.802089,19.048629,poland/slaski/bielsko-biala--ul.-partyzantow
4,Bielsko-Biała,"Bielsko-Biała, ul. Kossak-Szczuckiej 19, Poland",6535,49.813464,19.027318,poland/slaski/bielsko-biala--ul.-kossak-szczuc...


In [5]:
api_url = 'https://api.waqi.info'
api_token = '18d606fd3bbac19ab388244d380f3fe491ca2731'

In [33]:
params = ['pm25', 'pm10', 'o3', 'no2', 'so2', 'co']

In [34]:
current_measurements_df = pd.DataFrame(columns=['city', 'station', 'latitude', 'longitude', 'date', 'pm25', 'pm10', 'o3', 'no2', 'so2', 'co'])

In [35]:
for index, row in stations_df.iterrows():
    print(f'Szukanie pomiarów dla stacji: {index+1}')
    station_id = '@'+str(row['station_id'])
    current_api_url = api_url + f'/feed/{station_id}/?token={api_token}'
    response = requests.get(current_api_url)
    if response.status_code==200:
        if 'data' in response.json():
            response_data = response.json()['data']
            measurements = {key: None for key in params}
            for param in params:
                try:
                    measurements[param] = float(response_data['iaqi'][param]['v'])
                except (KeyError, ValueError, TypeError):
                    measurements[param] = None
            measurement_date = ''
            try:
                date_time_str = response_data['time']['iso']
                date_time_obj = datetime.fromisoformat(date_time_str)
                date_only = date_time_obj.date()
                measurement_date = date_only.strftime('%Y-%m-%d')
            except (KeyError, ValueError, Exception):
                measurement_date = None
            current_measurements_df.loc[len(current_measurements_df)] = [row['city'], row['station_name'], row['station_lat'], row['station_lon'],
                                                                         pd.to_datetime(measurement_date), measurements['pm25'],
                                                                         measurements['pm10'], measurements['o3'], measurements['no2'], measurements['so2'],
                                                                         measurements['co']]
    else:
        print('\tWystąpił błąd.')
    time.sleep(1)

Szukanie pomiarów dla stacji: 1
Szukanie pomiarów dla stacji: 2
Szukanie pomiarów dla stacji: 3
Szukanie pomiarów dla stacji: 4
Szukanie pomiarów dla stacji: 5
Szukanie pomiarów dla stacji: 6
Szukanie pomiarów dla stacji: 7
Szukanie pomiarów dla stacji: 8
Szukanie pomiarów dla stacji: 9
Szukanie pomiarów dla stacji: 10
Szukanie pomiarów dla stacji: 11
Szukanie pomiarów dla stacji: 12
Szukanie pomiarów dla stacji: 13
Szukanie pomiarów dla stacji: 14
Szukanie pomiarów dla stacji: 15
Szukanie pomiarów dla stacji: 16
Szukanie pomiarów dla stacji: 17
Szukanie pomiarów dla stacji: 18
Szukanie pomiarów dla stacji: 19
Szukanie pomiarów dla stacji: 20
Szukanie pomiarów dla stacji: 21
Szukanie pomiarów dla stacji: 22
Szukanie pomiarów dla stacji: 23
Szukanie pomiarów dla stacji: 24
Szukanie pomiarów dla stacji: 25
Szukanie pomiarów dla stacji: 26
Szukanie pomiarów dla stacji: 27
Szukanie pomiarów dla stacji: 28
Szukanie pomiarów dla stacji: 29
Szukanie pomiarów dla stacji: 30
Szukanie pomiarów d

In [36]:
current_measurements_df.head()

,city,station,latitude,longitude,date,pm25,pm10,o3,no2,so2,co
0,Jelenia Góra,"Jelenia Góra - Ogińskiego, Poland",50.913470,15.765659,2024-05-23,22.0,7.0,46.9,2.1,4.2,0.1
1,Biała Podlaska,"Biała Podlaska, Poland",52.038713,23.144503,2024-05-23,NaN,NaN,39.6,4.7,8.8,NaN
2,Białystok,"Białystok-Miejska, Poland",53.126815,23.156370,2023-11-13,63.0,NaN,NaN,4.7,NaN,NaN
3,Bielsko-Biała,"Bielsko-Biała, ul. Partyzantów, Poland",49.802089,19.048629,2024-05-23,51.0,NaN,NaN,4.9,NaN,0.1
4,Bielsko-Biała,"Bielsko-Biała, ul. Kossak-Szczuckiej 19, Poland",49.813464,19.027318,2024-05-23,NaN,8.0,20.5,3.7,0.6,NaN


In [53]:
current_measurements_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 123 entries, 0 to 122
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   city       123 non-null    object        
 1   station    123 non-null    object        
 2   latitude   123 non-null    float64       
 3   longitude  123 non-null    float64       
 4   date       112 non-null    datetime64[ns]
 5   pm25       75 non-null     float64       
 6   pm10       106 non-null    float64       
 7   o3         67 non-null     float64       
 8   no2        93 non-null     float64       
 9   so2        76 non-null     float64       
 10  co         66 non-null     float64       
dtypes: datetime64[ns](1), float64(8), object(2)
memory usage: 11.5+ KB


In [55]:
current_measurements_df_final = current_measurements_df.dropna(subset=['pm25', 'pm10', 'o3', 'no2', 'so2', 'co'])

In [56]:
current_measurements_df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36 entries, 0 to 119
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   city       36 non-null     object        
 1   station    36 non-null     object        
 2   latitude   36 non-null     float64       
 3   longitude  36 non-null     float64       
 4   date       34 non-null     datetime64[ns]
 5   pm25       36 non-null     float64       
 6   pm10       36 non-null     float64       
 7   o3         36 non-null     float64       
 8   no2        36 non-null     float64       
 9   so2        36 non-null     float64       
 10  co         36 non-null     float64       
dtypes: datetime64[ns](1), float64(8), object(2)
memory usage: 3.4+ KB
